In [ ]:
# Google drive connection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports and Dataset loading (Train-Test Split)

In [3]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import PredictionErrorDisplay
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from tensorflow.keras.layers import Input, Dense, Conv1D, Conv2D, Flatten, SimpleRNN, LSTM
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor

In [4]:
output_dir = "regression_results_modified_v6"
os.makedirs(output_dir, exist_ok=True)

In [5]:
dataset_path = "/content/drive/Shareddrives/Hardness_and_roughness/hardness_roughness.xlsx"
df = pd.read_excel(dataset_path)
X = df[['layer_thickness_(mm)', 'vaporizing_time_(min)', 'infill_rate_(%)']].values
y = df[['hardness_(Shore_D)', 'surface_roughness_(micrometer)']].values

#Machine Learning Regression Models

In [6]:
regression_models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "BayesianRidge": BayesianRidge(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "AdaBoost": AdaBoostRegressor(n_estimators=100, random_state=42),
    "ExtraTrees": ExtraTreesRegressor(n_estimators=100, random_state=42),
    "SVR": SVR(kernel='rbf', C=10, gamma='scale'),
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "XGBoost": XGBRegressor(verbosity=0, random_state=42),
    "Bagging": BaggingRegressor(n_estimators=50, random_state=42),
    "Stacking": StackingRegressor(
        estimators=[
            ('rf', RandomForestRegressor(n_estimators=50, random_state=42)),
            ('svr', SVR(kernel='rbf', C=10, gamma='scale')),
        ],
        final_estimator=Ridge()
    ),
    "MLP": MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
}

#1D CNN

In [7]:
def create_1d_cnn():
    input_layer = Input(shape=(3, 1), name='input_layer')
    x = Conv1D(32, 2, activation='relu')(input_layer)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_layer = Dense(2, name='regression_output')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='mse')
    return model

#2D CNN

In [8]:
def create_2d_cnn():
    input_layer = Input(shape=(3, 1, 1), name='input_layer')
    x = Conv2D(32, (2, 1), activation='relu')(input_layer)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_layer = Dense(2, name='regression_output')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='mse')
    return model

# RNN

In [9]:
def create_rnn():
    input_layer = Input(shape=(3, 1), name='input_layer')
    x = SimpleRNN(32, activation='relu', return_sequences=False)(input_layer)
    x = Dense(64, activation='relu')(x)
    output_layer = Dense(2, name='regression_output')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='mse')
    return model

# LSTM

In [10]:
def create_lstm():
    input_layer = Input(shape=(3, 1), name='input_layer')
    x = LSTM(32, activation='relu', return_sequences=False)(input_layer)
    x = Dense(64, activation='relu')(x)
    output_layer = Dense(2, name='regression_output')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='mse')
    return model

#Model Loading

In [11]:
nn_models = {
    'CNN_1D': create_1d_cnn,
    'CNN_2D': create_2d_cnn,
    'RNN': create_rnn,
    'LSTM': create_lstm
}

#Feature Extraction and ML Models

In [12]:
# --- sklearn-compatible Keras sequence head (single-output) ---
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

class KerasSeqHead(BaseEstimator, RegressorMixin):
    """
    Single-output sequence regressor so it works with MultiOutputRegressor.
    Expects input features of length `input_steps` (e.g., 32), reshaped to (N, steps, 1).
    kind: 'lstm' or 'rnn'
    """
    def __init__(self, kind='lstm', input_steps=32, units=32, dense_units=64,
                 lr=1e-3, epochs=200, batch_size=32, patience=15, verbose=0, seed=42):
        self.kind = kind
        self.input_steps = input_steps
        self.units = units
        self.dense_units = dense_units
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.patience = patience
        self.verbose = verbose
        self.seed = seed
        self._model = None

    def _build(self):
        tf.keras.utils.set_random_seed(self.seed)
        inp = tf.keras.Input(shape=(self.input_steps, 1), name="seq_input")
        if self.kind == 'lstm':
            x = layers.LSTM(self.units)(inp)   # default activations (tanh/sigmoid)
        else:
            x = layers.SimpleRNN(self.units)(inp)
        x = layers.Dense(self.dense_units, activation='relu')(x)
        out = layers.Dense(1)(x)  # SINGLE output -> compatible with MultiOutputRegressor
        m = Model(inp, out)
        m.compile(optimizer=Adam(self.lr), loss='mse')
        return m

    # scikit-learn API
    def get_params(self, deep=True):
        return {
            "kind": self.kind,
            "input_steps": self.input_steps,
            "units": self.units,
            "dense_units": self.dense_units,
            "lr": self.lr,
            "epochs": self.epochs,
            "batch_size": self.batch_size,
            "patience": self.patience,
            "verbose": self.verbose,
            "seed": self.seed,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y).reshape(-1, 1)  # ensure (N,1) for single target
        # reshape features to (N, steps, 1)
        if X.ndim == 2 and X.shape[1] == self.input_steps:
            X = X.reshape(-1, self.input_steps, 1)
        elif X.ndim != 3 or X.shape[1:] != (self.input_steps, 1):
            raise ValueError(f"X must be (N,{self.input_steps}) or (N,{self.input_steps},1); got {X.shape}")

        self._model = self._build()
        es = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=self.patience,
                                              restore_best_weights=True)
        self._model.fit(X, y,
                        epochs=self.epochs,
                        batch_size=self.batch_size,
                        verbose=self.verbose,
                        callbacks=[es])
        return self

    def predict(self, X):
        X = np.asarray(X)
        if X.ndim == 2 and X.shape[1] == self.input_steps:
            X = X.reshape(-1, self.input_steps, 1)
        return self._model.predict(X, verbose=0).ravel()  # (N,)

In [13]:
def create_cnn_feature_model():
    input_layer = tf.keras.Input(shape=(3, 1))
    x = layers.Conv1D(32, 2, activation='relu')(input_layer)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    feature_output = layers.Dense(32, activation='relu', name="feature_layer")(x)
    final_output = layers.Dense(2, name="regression_output")(feature_output)

    model = Model(inputs=input_layer, outputs=final_output)
    model.compile(optimizer='adam', loss='mse')
    return model

# ========== ML Models ==========
feature_models = {
    'Feature_RF': MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42)),
    'Feature_Ridge': MultiOutputRegressor(Ridge()),
    'Feature_SVR': MultiOutputRegressor(SVR()),
    'Feature_XGBoost': MultiOutputRegressor(XGBRegressor(verbosity=0, random_state=42)),
    'Feature_MLP': MultiOutputRegressor(MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42)),
    'Feature_KNN': MultiOutputRegressor(KNeighborsRegressor(n_neighbors=5)),
    'Feature_DecisionTree': MultiOutputRegressor(DecisionTreeRegressor(random_state=42)),
    'Feature_LinearRegression': MultiOutputRegressor(LinearRegression()),
    'Feature_BayesianRidge': MultiOutputRegressor(BayesianRidge()),
    'Feature_GradientBoosting': MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, random_state=42)),
    'Feature_AdaBoost': MultiOutputRegressor(AdaBoostRegressor(n_estimators=100, random_state=42)),
    'Feature_ExtraTrees': MultiOutputRegressor(ExtraTreesRegressor(n_estimators=100, random_state=42)),
    'Feature_Bagging': MultiOutputRegressor(BaggingRegressor(n_estimators=50, random_state=42)),
    'Feature_Stacking': MultiOutputRegressor(StackingRegressor(
        estimators=[
            ('rf', RandomForestRegressor(n_estimators=50, random_state=42)),
            ('svr', SVR(kernel='rbf', C=10, gamma='scale')),
        ],
        final_estimator=Ridge()
    ))
}

feature_models.update({
    'Feature_LSTM': MultiOutputRegressor(
        KerasSeqHead(kind='lstm', input_steps=32, epochs=300, verbose=0)
    ),
    'Feature_RNN': MultiOutputRegressor(
        KerasSeqHead(kind='rnn',  input_steps=32, epochs=300, verbose=0)
    ),
})

#K-fold (5) Dataset Separation

In [14]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
results = []
all_fold_outputs = {}

In [15]:
colors = ['blue', 'orange']
output_names = ['hardness', 'surface_roughness']

#Classic Machine Learning Models on K-Folds and Graph Outputs

In [16]:
for name, model in regression_models.items():
    mse_scores = []
    r2_scores = []

    # Wrap in MultiOutputRegressor if needed
    if name not in ["RandomForest", "ExtraTrees", "XGBoost"]:
        model = MultiOutputRegressor(model)

    all_fold_outputs[name] = []

    plt.figure(figsize=(15, 10))
    plt.suptitle(f'{name} Model - K-Fold Regression Accuracy (k=5)', fontsize=16)

    fold_iter = tqdm(enumerate(kf.split(X), 1), total=kf.get_n_splits(), desc=f"Training {name}")

    for fold, (train_index, test_index) in fold_iter:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Scaling
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)

        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_test_scaled = scaler_y.transform(y_test)

        try:
            model.fit(X_train_scaled, y_train_scaled)
            y_pred_scaled = model.predict(X_test_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled)
            y_test_actual = scaler_y.inverse_transform(y_test_scaled)

            mse = mean_squared_error(y_test_actual, y_pred)
            r2 = r2_score(y_test_actual, y_pred)

            mse_scores.append(mse)
            r2_scores.append(r2)

            # === NEW: Store fold outputs ===
            all_fold_outputs[name].append({
                'fold': fold,
                'y_true': y_test_actual,
                'y_pred': y_pred,
                'r2': r2,
                'mse': mse
            })

            # Scatter Plot - True vs Predicted
            plt.subplot(2, 3, fold)
            plt.scatter(y_test_actual[:, 0], y_pred[:, 0],
                        alpha=0.6, color='blue', label='Hardness')
            plt.scatter(y_test_actual[:, 1], y_pred[:, 1],
                        alpha=0.6, color='orange', label='Surface Roughness')

            min_val = min(y_test_actual.min(), y_pred.min())
            max_val = max(y_test_actual.max(), y_pred.max())

            plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
            plt.title(f'Fold {fold} (R²={r2:.2f})')
            plt.xlabel('True Values')
            plt.ylabel('Predictions')
            plt.xlim(min_val, max_val)
            plt.ylim(min_val, max_val)
            plt.legend()
            plt.xticks(np.linspace(min_val, max_val, 5))
            plt.yticks(np.linspace(min_val, max_val, 5))

            # Line Plot with Errors
            plt.figure(figsize=(14, 10))
            plt.subplot(2, 1, 1)
            plt.plot(range(len(y_test_actual)), y_test_actual[:, 0], 'o-', label='True Hardness', color='blue')
            plt.plot(range(len(y_test_actual)), y_pred[:, 0], 'x--', label='Predicted Hardness', color='cyan')
            plt.plot(range(len(y_test_actual)), y_test_actual[:, 1], 'o-', label='True Roughness', color='orange')
            plt.plot(range(len(y_test_actual)), y_pred[:, 1], 'x--', label='Predicted Roughness', color='red')
            plt.title(f'{name} - Fold {fold}\nTrue vs Predicted Values', pad=20)
            plt.ylabel('Values')
            plt.legend()
            plt.grid(True, alpha=0.3)

            plt.subplot(2, 1, 2)
            for i in range(len(y_test_actual)):
                plt.plot([i, i], [0, y_test_actual[i, 0] - y_pred[i, 0]], color='blue', alpha=0.5, linewidth=2)
                plt.scatter(i, y_test_actual[i, 0] - y_pred[i, 0], color='blue', s=80, alpha=0.8,
                            label='Hardness Error' if i == 0 else "")
                plt.plot([i + 0.1, i + 0.1], [0, y_test_actual[i, 1] - y_pred[i, 1]], color='orange', alpha=0.5, linewidth=2)
                plt.scatter(i + 0.1, y_test_actual[i, 1] - y_pred[i, 1], color='orange', s=80, alpha=0.8,
                            label='Surface Roughness Error' if i == 0 else "")

            plt.title('Prediction Errors', pad=10)
            plt.xlabel('Sample Index')
            plt.ylabel('Error Magnitude (True - Predicted)')
            plt.axhline(0, color='black', linestyle='-', linewidth=1)
            plt.legend()
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'{name}_Fold{fold}_line_plot_errors_en.png'), dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            print(f"{name} - Fold {fold} error: {str(e)}")
            continue

    # Summary subplot
    plt.subplot(2, 3, fold + 1 if fold < 6 else 6)
    plt.text(0.1, 0.7, f"Average MSE: {np.mean(mse_scores):.4f}\n"
                       f"MSE Std: {np.std(mse_scores):.4f}\n"
                       f"Average R²: {np.mean(r2_scores):.4f}\n"
                       f"R² Std: {np.std(r2_scores):.4f}\n"
                       f"Successful Folds: {len(mse_scores)}/{kf.get_n_splits()}",
             fontsize=12)
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{name}_regression_summary_en.png'), dpi=300, bbox_inches='tight')
    plt.close()

    if mse_scores:
        results.append({
            'Model': name,
            'Avg MSE': round(np.mean(mse_scores), 4),
            'MSE Std': round(np.std(mse_scores), 4),
            'Avg R²': round(np.mean(r2_scores), 4),
            'R² Std': round(np.std(r2_scores), 4),
            'Successful Folds': f"{len(mse_scores)}/{kf.get_n_splits()}"
        })

Training MLP:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
Training MLP:  20%|██        | 1/5 [00:03<00:14,  3.62s/it]/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
Training MLP:  40%|████      | 2/5 [00:06<00:09,  3.07s/it]/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
Training MLP:  60%|██████    | 3/5 [00:09<00:06,  3.33s/it]/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron

#Neural Networks Models on K-Folds and Graph Outputs

In [17]:
for name, model_fn in nn_models.items():
    mse_scores = []
    r2_scores = []
    all_fold_outputs[name] = []

    plt.figure(figsize=(15, 10))
    plt.suptitle(f'{name} Model - K-Fold Regression Accuracy (k={k})', fontsize=16)

    fold_iter = tqdm(enumerate(kf.split(X), 1), total=k, desc=f"Training {name}")

    for fold, (train_index, test_index) in fold_iter:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Scaling
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)

        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_test_scaled = scaler_y.transform(y_test)

        # Reshape input
        if name in ['CNN_1D', 'RNN', 'LSTM']:
            X_train_reshaped = X_train_scaled.reshape(-1, 3, 1)
            X_test_reshaped = X_test_scaled.reshape(-1, 3, 1)
        elif name == 'CNN_2D':
            X_train_reshaped = X_train_scaled.reshape(-1, 3, 1, 1)
            X_test_reshaped = X_test_scaled.reshape(-1, 3, 1, 1)
        else:
            X_train_reshaped = X_train_scaled
            X_test_reshaped = X_test_scaled

        try:
            model = model_fn()
            model.fit(
                X_train_reshaped, y_train_scaled,
                epochs=100,
                batch_size=16,
                verbose=0
            )

            y_pred_scaled = model.predict(X_test_reshaped)
            y_pred = scaler_y.inverse_transform(y_pred_scaled)
            y_test_actual = scaler_y.inverse_transform(y_test_scaled)

            mse = mean_squared_error(y_test_actual, y_pred)
            r2 = r2_score(y_test_actual, y_pred)

            mse_scores.append(mse)
            r2_scores.append(r2)

            # === Store fold outputs ===
            all_fold_outputs[name].append({
                'fold': fold,
                'y_true': y_test_actual,
                'y_pred': y_pred,
                'r2': r2,
                'mse': mse
            })

            # Fold scatter plot
            plt.subplot(2, 3, fold)
            plt.scatter(y_test_actual[:, 0], y_pred[:, 0], alpha=0.6, color='blue', label='Hardness')
            plt.scatter(y_test_actual[:, 1], y_pred[:, 1], alpha=0.6, color='orange', label='Surface Roughness')
            min_val = min(y_test_actual.min(), y_pred.min())
            max_val = max(y_test_actual.max(), y_pred.max())
            plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
            plt.title(f'Fold {fold} (R²={r2:.2f})')
            plt.xlabel('True Values')
            plt.ylabel('Predictions')
            plt.xlim(min_val, max_val)
            plt.ylim(min_val, max_val)
            plt.legend()
            plt.xticks(np.linspace(min_val, max_val, 5))
            plt.yticks(np.linspace(min_val, max_val, 5))

            # Line plot with errors
            plt.figure(figsize=(14, 10))
            plt.subplot(2, 1, 1)
            plt.plot(range(len(y_test_actual)), y_test_actual[:, 0], 'o-', label='True Hardness', color='blue')
            plt.plot(range(len(y_test_actual)), y_pred[:, 0], 'x--', label='Predicted Hardness', color='cyan')
            plt.plot(range(len(y_test_actual)), y_test_actual[:, 1], 'o-', label='True Roughness', color='orange')
            plt.plot(range(len(y_test_actual)), y_pred[:, 1], 'x--', label='Predicted Roughness', color='red')
            plt.title(f'{name} - Fold {fold}\nTrue vs Predicted Values', pad=20)
            plt.ylabel('Values')
            plt.legend()
            plt.grid(True, alpha=0.3)

            plt.subplot(2, 1, 2)
            for i in range(len(y_test_actual)):
                plt.plot([i, i], [0, y_test_actual[i, 0] - y_pred[i, 0]], color='blue', alpha=0.5, linewidth=2)
                plt.scatter(i, y_test_actual[i, 0] - y_pred[i, 0], color='blue', s=80, alpha=0.8,
                            label='Hardness Error' if i == 0 else "")
                plt.plot([i + 0.1, i + 0.1], [0, y_test_actual[i, 1] - y_pred[i, 1]],
                         color='orange', alpha=0.5, linewidth=2)
                plt.scatter(i + 0.1, y_test_actual[i, 1] - y_pred[i, 1],
                            color='orange', s=80, alpha=0.8,
                            label='Surface Roughness Error' if i == 0 else "")
            plt.title('Prediction Errors', pad=10)
            plt.xlabel('Sample Index')
            plt.ylabel('Error Magnitude (True - Predicted)')
            plt.axhline(0, color='black', linestyle='-', linewidth=1)
            plt.legend()
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'{name}_Fold{fold}_line_plot_errors_en.png'), dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            print(f"{name} - Fold {fold} error: {str(e)}")
            continue

    # Summary subplot
    plt.subplot(2, 3, fold + 1 if fold < 6 else 6)
    plt.text(0.1, 0.7, f"Average MSE: {np.mean(mse_scores):.4f}\n"
                       f"MSE Std: {np.std(mse_scores):.4f}\n"
                       f"Average R²: {np.mean(r2_scores):.4f}\n"
                       f"R² Std: {np.std(r2_scores):.4f}\n"
                       f"Successful Folds: {len(mse_scores)}/{k}",
             fontsize=12)
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{name}_regression_summary_en.png'), dpi=300, bbox_inches='tight')
    plt.close()

    if mse_scores:
        results.append({
            'Model': name,
            'Avg MSE': round(np.mean(mse_scores), 4),
            'MSE Std': round(np.std(mse_scores), 4),
            'Avg R²': round(np.mean(r2_scores), 4),
            'R² Std': round(np.std(r2_scores), 4),
            'Successful Folds': f"{len(mse_scores)}/{k}"
        })

Training CNN_1D:   0%|          | 0/5 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


Training CNN_1D:  20%|██        | 1/5 [00:11<00:45, 11.27s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


Training CNN_1D:  40%|████      | 2/5 [00:21<00:31, 10.47s/it]

...

Training LSTM:  60%|██████    | 3/5 [00:36<00:24, 12.22s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step


Training LSTM:  80%|████████  | 4/5 [00:48<00:11, 11.85s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step


Training LSTM: 100%|██████████| 5/5 [01:00<00:00, 12.02s/it]


#Feature Extraction Models on K-Folds and Graph Outputs

In [18]:
for name, model in feature_models.items():
    mse_scores = []
    r2_scores = []
    all_fold_outputs[name] = []

    plt.figure(figsize=(15, 10))
    plt.suptitle(f'{name} Model - K-Fold Regression Accuracy (k={k})', fontsize=16)

    fold_iter = tqdm(enumerate(kf.split(X), 1), total=k, desc=f"Training {name}")

    for fold, (train_index, test_index) in fold_iter:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Scaling
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)

        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_test_scaled = scaler_y.transform(y_test)

        # Reshape for CNN input
        X_train_reshaped = X_train_scaled.reshape(-1, 3, 1)
        X_test_reshaped = X_test_scaled.reshape(-1, 3, 1)

        try:
            # Train CNN for feature extraction
            cnn_model = create_cnn_feature_model()
            cnn_model.fit(X_train_reshaped, y_train_scaled, epochs=100, batch_size=16, verbose=0)

            # Extract features
            feature_extractor = Model(inputs=cnn_model.input,
                                      outputs=cnn_model.get_layer("feature_layer").output)
            X_train_feat = feature_extractor.predict(X_train_reshaped)
            X_test_feat = feature_extractor.predict(X_test_reshaped)

            # Train ML model
            model.fit(X_train_feat, y_train_scaled)
            y_pred_scaled = model.predict(X_test_feat)
            y_pred = scaler_y.inverse_transform(y_pred_scaled)
            y_test_actual = scaler_y.inverse_transform(y_test_scaled)

            if np.any(np.isnan(y_pred)):
                print(f"{name} - Fold {fold} error: NaNs in predictions")
                continue

            # Metrics
            mse = mean_squared_error(y_test_actual, y_pred)
            r2 = r2_score(y_test_actual, y_pred)

            mse_scores.append(mse)
            r2_scores.append(r2)

            # === Store fold outputs ===
            all_fold_outputs[name].append({
                'fold': fold,
                'y_true': y_test_actual,
                'y_pred': y_pred,
                'r2': r2,
                'mse': mse
            })

            # Fold scatter plot
            plt.subplot(2, 3, fold)
            plt.scatter(y_test_actual[:, 0], y_pred[:, 0], alpha=0.6, color='blue', label='Hardness')
            plt.scatter(y_test_actual[:, 1], y_pred[:, 1], alpha=0.6, color='orange', label='Surface Roughness')

            min_val = min(y_test_actual.min(), y_pred.min())
            max_val = max(y_test_actual.max(), y_pred.max())
            plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)

            plt.title(f'Fold {fold} (R²={r2:.2f})')
            plt.xlabel('True Values')
            plt.ylabel('Predictions')
            plt.xlim(min_val, max_val)
            plt.ylim(min_val, max_val)
            plt.legend()
            plt.xticks(np.linspace(min_val, max_val, 5))
            plt.yticks(np.linspace(min_val, max_val, 5))

            # Line plot with errors
            plt.figure(figsize=(14, 10))

            plt.subplot(2, 1, 1)
            plt.plot(range(len(y_test_actual)), y_test_actual[:, 0], 'o-', label='True Hardness', color='blue')
            plt.plot(range(len(y_test_actual)), y_pred[:, 0], 'x--', label='Predicted Hardness', color='cyan')
            plt.plot(range(len(y_test_actual)), y_test_actual[:, 1], 'o-', label='True Roughness', color='orange')
            plt.plot(range(len(y_test_actual)), y_pred[:, 1], 'x--', label='Predicted Roughness', color='red')
            plt.title(f'{name} - Fold {fold}\nTrue vs Predicted Values', pad=20)
            plt.ylabel('Values')
            plt.legend()
            plt.grid(True, alpha=0.3)

            plt.subplot(2, 1, 2)
            for i in range(len(y_test_actual)):
                plt.plot([i, i], [0, y_test_actual[i, 0] - y_pred[i, 0]], color='blue', alpha=0.5)
                plt.scatter(i, y_test_actual[i, 0] - y_pred[i, 0], color='blue', s=80, alpha=0.8,
                            label='Hardness Error' if i == 0 else "")
                plt.plot([i + 0.1, i + 0.1], [0, y_test_actual[i, 1] - y_pred[i, 1]], color='orange', alpha=0.5)
                plt.scatter(i + 0.1, y_test_actual[i, 1] - y_pred[i, 1], color='orange', s=80, alpha=0.8,
                            label='Roughness Error' if i == 0 else "")

            plt.title('Prediction Errors', pad=10)
            plt.xlabel('Sample Index')
            plt.ylabel('Error Magnitude (True - Predicted)')
            plt.axhline(0, color='black', linestyle='-', linewidth=1)
            plt.legend()
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'{name}_Fold{fold}_line_plot_errors_en.png'), dpi=300, bbox_inches='tight')
            plt.close()

        except Exception as e:
            print(f"{name} - Fold {fold} error: {e}")
            continue

    # Summary statistics plot
    plt.subplot(2, 3, 6)
    plt.text(0.1, 0.7, f"Average MSE: {np.mean(mse_scores):.4f}\n"
                       f"MSE Std: {np.std(mse_scores):.4f}\n"
                       f"Average R²: {np.mean(r2_scores):.4f}\n"
                       f"R² Std: {np.std(r2_scores):.4f}\n"
                       f"Successful Folds: {len(mse_scores)}/{k}",
             fontsize=12)
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{name}_regression_summary_en.png'), dpi=300, bbox_inches='tight')
    plt.close()

    if mse_scores:
        results.append({
            'Model': name,
            'Avg MSE': float(round(np.mean(mse_scores), 4)),
            'MSE Std': float(round(np.std(mse_scores), 4)),
            'Avg R²': float(round(np.mean(r2_scores), 4)),
            'R² Std': float(round(np.std(r2_scores), 4)),
            'Successful Folds': f"{len(mse_scores)}/{k}"
        })

Training Feature_RF:   0%|          | 0/5 [00:00<?, ?it/s]

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


Training Feature_RF:  20%|██        | 1/5 [00:11<00:44, 11.22s/it]

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


Training Feature_RF:  40%|████      | 2/5 [00:22<00:34, 11.37s/it]

...

Training Feature_RNN:  60%|██████    | 3/5 [02:41<01:51, 55.52s/it]

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


Training Feature_RNN:  80%|████████  | 4/5 [03:16<00:47, 47.78s/it]

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


Training Feature_RNN: 100%|██████████| 5/5 [04:12<00:00, 50.45s/it]


#Show Results

In [19]:
# Create results DataFrame
results_df = pd.DataFrame(results)
results_df.sort_values(by='Avg R²', ascending=False, inplace=True)

# Print results
print("\nPerformance Comparison of Models (sorted by Avg R²):")
print(results_df.to_string(index=False))

# Save as CSV in the output folder
results_csv_path = os.path.join(output_dir, "regression_model_comparison.csv")
results_df.to_csv(results_csv_path, index=False)


Performance Comparison of Models (sorted by Avg R²):
                   Model  Avg MSE  MSE Std  Avg R²  R² Std Successful Folds
             Feature_SVR   2.1658   0.5670  0.9595  0.0123              5/5
           Feature_Ridge   2.0207   0.4922  0.9589  0.0076              5/5
        GradientBoosting   2.2007   0.7024  0.9580  0.0090              5/5
                     RNN   1.9772   0.4791  0.9574  0.0099              5/5
              Feature_RF   2.1727   0.3912  0.9572  0.0106              5/5
   Feature_BayesianRidge   2.1818   0.5143  0.9568  0.0088              5/5
Feature_LinearRegression   2.2527   0.6614  0.9565  0.0108              5/5
                  CNN_2D   1.8658   0.3931  0.9559  0.0111              5/5
                     SVR   2.0916   0.5130  0.9547  0.0085              5/5
                 Bagging   2.4673   0.1428  0.9544  0.0114              5/5
        Feature_Stacking   2.4742   0.7918  0.9543  0.0085              5/5
             Feature_RNN   2.2176 

In [20]:
import pandas as pd

# ---------- Build fold-wise table ----------
flattened_data = []
for model_name, folds in all_fold_outputs.items():
    mse_values, r2_values = [], []
    fold_dict = {'Model': model_name}

    for i, fold in enumerate(folds, 1):
        mse = fold['mse']
        r2 = fold['r2']
        mse_values.append(mse)
        r2_values.append(r2)
        fold_dict[f'Fold {i} R²'] = round(r2, 4)
        fold_dict[f'Fold {i} MSE'] = round(mse, 4)

    if len(mse_values) > 0:
        fold_dict['Avg MSE'] = round(sum(mse_values) / len(mse_values), 4)
        fold_dict['Avg R²']  = round(sum(r2_values) / len(r2_values), 4)
    else:
        fold_dict['Avg MSE'] = None
        fold_dict['Avg R²']  = None

    flattened_data.append(fold_dict)

# DataFrame with all models' fold-wise metrics
df_folds_separated = pd.DataFrame(flattened_data)

# ---------- Top 5 and Worst 2 by Avg R² ----------
df_sorted = df_folds_separated.sort_values(by='Avg R²', ascending=False)

df_top5_separated = df_sorted.head(5)
n_worst = min(2, len(df_sorted))
df_worst2_separated = df_sorted.tail(n_worst).sort_values(by='Avg R²', ascending=True)

# ---------- Display ----------
pd.set_option('display.max_columns', None)

print("\n=== TOP 5 MODELS (by Avg R²) ===")
print(df_top5_separated.to_string(index=False))

print("\n=== WORST 2 MODELS (by Avg R²) ===")
print(df_worst2_separated.to_string(index=False))

# ---------- Save CSVs ----------
top5_csv = os.path.join(output_dir, "top5_model_fold_results.csv")
worst2_csv = os.path.join(output_dir, "worst2_model_fold_results.csv")
all_csv = os.path.join(output_dir, "all_model_fold_results.csv")

df_top5_separated.to_csv(top5_csv, index=False)
df_worst2_separated.to_csv(worst2_csv, index=False)
df_folds_separated.to_csv(all_csv, index=False)


=== TOP 5 MODELS (by Avg R²) ===
           Model  Fold 1 R²  Fold 1 MSE  Fold 2 R²  Fold 2 MSE  Fold 3 R²  Fold 3 MSE  Fold 4 R²  Fold 4 MSE  Fold 5 R²  Fold 5 MSE  Avg MSE  Avg R²
     Feature_SVR     0.9432      2.3926     0.9750      2.0604     0.9725      1.5481     0.9552      3.1351     0.9515      1.6928   2.1658  0.9595
   Feature_Ridge     0.9506      1.7339     0.9657      2.1259     0.9697      1.6356     0.9571      2.9402     0.9515      1.6682   2.0207  0.9589
GradientBoosting     0.9498      1.7380     0.9715      2.1283     0.9662      2.0053     0.9524      3.5515     0.9503      1.5804   2.2007  0.9580
             RNN     0.9483      1.9438     0.9731      1.8673     0.9619      1.7815     0.9582      2.8673     0.9455      1.4259   1.9772  0.9574
      Feature_RF     0.9403      2.1147     0.9737      1.9813     0.9596      2.5173     0.9570      2.6737     0.9552      1.5767   2.1727  0.9572

=== WORST 2 MODELS (by Avg R²) ===
           Model  Fold 1 R²  Fold 1 

In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt

output_dir_result = output_dir + "_result"
os.makedirs(output_dir_result, exist_ok=True)

plt.rcParams.update({
    'xtick.labelsize': 20,
    'ytick.labelsize': 20
})

# --- Step 1: pick best model ---
model_avg_r2 = {
    model: np.mean([fold['r2'] for fold in folds])
    for model, folds in all_fold_outputs.items()
}
best_model = max(model_avg_r2, key=model_avg_r2.get)
best_folds = all_fold_outputs[best_model]

# --- Step 2: make SEPARATE figures per fold and per target ---
for i, fold_data in enumerate(best_folds, start=1):
    y_true = fold_data['y_true']
    y_pred = fold_data['y_pred']

    # -----------------------
    # Graph 1A: True vs Predicted (Hardness)
    # -----------------------
    fig1, ax1 = plt.subplots(figsize=(12, 6))
    fig1.suptitle(f'Fold {i} - Hardness', fontsize=30, weight='bold')

    ax1.plot(y_true[:, 0], 'o-', label='True Hardness', color='navy', marker='o')
    ax1.plot(y_pred[:, 0], 's--', label='Predicted Hardness', color='darkorange', marker='s')

    ax1.set_xlabel('Sample Index', fontsize=22)
    ax1.set_ylabel('Hardness (Shore D)', fontsize=22)
    ax1.set_title('True vs Predicted - Hardness', fontsize=26, weight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=2,
               fontsize=20, borderaxespad=2.0)

    fig1.tight_layout(rect=[0, 0.05, 0.9, 0.95])
    path1 = os.path.join(output_dir_result, f'{best_model}_Fold{i}_true_vs_pred_hardness.png')
    fig1.savefig(path1, dpi=300, bbox_inches='tight')
    plt.close(fig1)
    print(f"Saved: {path1}")

    # -----------------------
    # Graph 1B: True vs Predicted (Roughness)
    # -----------------------
    fig2, ax2 = plt.subplots(figsize=(12, 6))
    fig2.suptitle(f'Fold {i} - Surface Roughness', fontsize=30, weight='bold')

    ax2.plot(y_true[:, 1], 'o-', label='True Roughness', color='limegreen', marker='o')
    ax2.plot(y_pred[:, 1], 's--', label='Predicted Roughness', color='crimson', marker='s')

    ax2.set_xlabel('Sample Index', fontsize=22)
    ax2.set_ylabel('Surface Roughness (μm)', fontsize=22)
    ax2.set_title('True vs Predicted - Roughness', fontsize=26, weight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=2,
               fontsize=20, borderaxespad=2.0)

    fig2.tight_layout(rect=[0, 0.05, 0.9, 0.95])
    path2 = os.path.join(output_dir_result, f'{best_model}_Fold{i}_true_vs_pred_roughness.png')
    fig2.savefig(path2, dpi=300, bbox_inches='tight')
    plt.close(fig2)
    print(f"Saved: {path2}")

    # -----------------------
    # Graph 2A: Error Plot (Hardness)
    # -----------------------
    fig3, ax3 = plt.subplots(figsize=(12, 6))
    error_hardness = y_true[:, 0] - y_pred[:, 0]

    ax3.vlines(range(len(y_true)), 0, error_hardness, color='navy', alpha=0.7, linewidth=2)
    ax3.scatter(range(len(y_true)), error_hardness, color='deepskyblue',
                s=90, edgecolor='k', label='Hardness Error')
    ax3.axhline(0, color='black', linestyle='-', linewidth=1)

    ax3.set_xlabel('Sample Index', fontsize=22)
    ax3.set_ylabel('Error Magnitude', fontsize=22)
    ax3.set_title('Prediction Error - Hardness', fontsize=26, weight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=1,
               fontsize=20, borderaxespad=2.0)

    fig3.tight_layout(rect=[0, 0.05, 0.9, 0.95])
    path3 = os.path.join(output_dir_result, f'{best_model}_Fold{i}_error_hardness.png')
    fig3.savefig(path3, dpi=300, bbox_inches='tight')
    plt.close(fig3)
    print(f"Saved: {path3}")

    # -----------------------
    # Graph 2B: Error Plot (Roughness)
    # -----------------------
    fig4, ax4 = plt.subplots(figsize=(12, 6))
    error_roughness = y_true[:, 1] - y_pred[:, 1]

    ax4.vlines(range(len(y_true)), 0, error_roughness, color='darkorange', alpha=0.7, linewidth=2)
    ax4.scatter(range(len(y_true)), error_roughness, color='orangered',
                s=90, edgecolor='k', label='Roughness Error')
    ax4.axhline(0, color='black', linestyle='-', linewidth=1)

    ax4.set_xlabel('Sample Index', fontsize=22)
    ax4.set_ylabel('Error Magnitude', fontsize=22)
    ax4.set_title('Prediction Error - Roughness', fontsize=26, weight='bold')
    ax4.grid(True, alpha=0.3)
    ax4.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=1,
               fontsize=20, borderaxespad=2.0)

    fig4.tight_layout(rect=[0, 0.05, 0.9, 0.95])
    path4 = os.path.join(output_dir_result, f'{best_model}_Fold{i}_error_roughness.png')
    fig4.savefig(path4, dpi=300, bbox_inches='tight')
    plt.close(fig4)
    print(f"Saved: {path4}")

/tmp/ipython-input-1426982594.py:32: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o-" (-> marker='o'). The keyword argument will take precedence.
  ax1.plot(y_true[:, 0], 'o-', label='True Hardness', color='navy', marker='o')
/tmp/ipython-input-1426982594.py:33: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "s--" (-> marker='s'). The keyword argument will take precedence.
  ax1.plot(y_pred[:, 0], 's--', label='Predicted Hardness', color='darkorange', marker='s')


Saved: regression_results_modified_v6_result/Feature_SVR_Fold1_true_vs_pred_hardness.png


/tmp/ipython-input-1426982594.py:54: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o-" (-> marker='o'). The keyword argument will take precedence.
  ax2.plot(y_true[:, 1], 'o-', label='True Roughness', color='limegreen', marker='o')
/tmp/ipython-input-1426982594.py:55: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "s--" (-> marker='s'). The keyword argument will take precedence.
  ax2.plot(y_pred[:, 1], 's--', label='Predicted Roughness', color='crimson', marker='s')


Saved: regression_results_modified_v6_result/Feature_SVR_Fold1_true_vs_pred_roughness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold1_error_hardness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold1_error_roughness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold2_true_vs_pred_hardness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold2_true_vs_pred_roughness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold2_error_hardness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold2_error_roughness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold3_true_vs_pred_hardness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold3_true_vs_pred_roughness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold3_error_hardness.png
Saved: regression_results_modified_v6_result/Feature_SVR_Fold3_error_roughness.png
Saved: regression_results_modified_v6_result/Feature_SVR_

#Copy the Result Folder to The Drive

In [22]:
source_folder = output_dir
destination_folder = f"/content/drive/Shareddrives/Hardness_and_roughness/{output_dir}"

# Copy the entire folder and its contents
shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)

'/content/drive/Shareddrives/Hardness_and_roughness/regression_results_modified_v6'

In [23]:
source_folder = output_dir_result
destination_folder = f"/content/drive/Shareddrives/Hardness_and_roughness/{output_dir_result}"

# Copy the entire folder and its contents
shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)

'/content/drive/Shareddrives/Hardness_and_roughness/regression_results_modified_v6_result'

In [24]:
output_dir_result = "/content/regression_results_modified_v6_result"
source_folder = output_dir_result
destination_folder = f"/content/drive/Shareddrives/Hardness_and_roughness/regression_results_modified_v6_result"

# Copy the entire folder and its contents
shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)

'/content/drive/Shareddrives/Hardness_and_roughness/regression_results_modified_v6_result'